# Final Presentation ACDC

In this notebook we will walk through examples of this package. The following points are covered: 
- Find downstream vertices
- Find alternative edges
- Handle PGM input format
- Raise errors if the data is incorrect. 
- Aggreate the power flow results in tables. 
- Show tables with data of powerflow calculations. 
- Calculate EV penetration levels. 
- Calculate optimal tap positions. 
- Do N-1 Calculations. 

# Input dataset

We create an input dataset by using the following parameters: 
- vertex_ids
- edge_ids
- edge_vertex_id_pairs
- edge_enabled
- source_vertex_id

In [1]:
vertex_ids = [0, 2, 4, 6, 10]  # All unique vertex ids
edge_ids = [1, 3, 5, 7, 8, 9]  # All unique edge ids
edge_vertex_id_pairs = [
        (0, 2),  # edge 1
        (0, 4),  # edge 3
        (0, 6),  # edge 5
        (2, 4),  # edge 7
        (4, 6),  # edge 8
        (2, 10),  # edge 9
    ]
edge_enabled = [True, True, True, False, False, True]  # Whether each edge is enabled or disabled
source_vertex_id = 0  # ID of the source vertex

This will give the following graph: 

In [2]:
'''      
vertex_0 (source) --edge_1(enabled)-- vertex_2 --edge_9(enabled)-- vertex_10
                 |                               |
                 |                           edge_7(disabled)
                 |                               |
                 -----------edge_3(enabled)-- vertex_4
                 |                               |
                 |                           edge_8(disabled)
                 |                               |
                 -----------edge_5(enabled)-- vertex_6
'''

'      \nvertex_0 (source) --edge_1(enabled)-- vertex_2 --edge_9(enabled)-- vertex_10\n                 |                               |\n                 |                           edge_7(disabled)\n                 |                               |\n                 -----------edge_3(enabled)-- vertex_4\n                 |                               |\n                 |                           edge_8(disabled)\n                 |                               |\n                 -----------edge_5(enabled)-- vertex_6\n'

# Validation
This graph is tested for the following conditions: 
1. vertex_ids and edge_ids should be unique
2. edge_vertex_id_pairs should have the same length as edge_ids
3. edge_vertex_id_pairs should contain valid vertex ids
4. edge_enabled should have the same length as edge_ids
5. source_vertex_id should be a valid vertex id
6. The graph should not contain cycles
7. The graph should be fully connected
8. Multiple edges should not connect the same two vertices. 